<a href="https://colab.research.google.com/github/WKhisa/Feature-Engineering-with-Python/blob/main/Feature_Engineering_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Research Question**
# Can we build a model that predicts an accurate delivery time, from picking up a package arriving at the final destination?

**Success Metrics**
If model can accurately predict arrival time



# Load Data



In [2]:
#Install packages
!pip install mlrose

  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post7-py3-none-any.whl size=2951 sha256=0a3b501d9b8f43286ccf4d46bcf328dc91709696c00dfa5db5732ea022bae571
  Stored in directory: /root/.cache/pip/wheels/c8/9c/85/72901eb50bc4bc6e3b2629378d172384ea3dfd19759c77fd2c
Successfully built sklearn


In [3]:
# Importing our libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
pd.set_option('display.max_columns', None)
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
import joblib
sys.modules['sklearn.externals.joblib'] = joblib
from mlxtend.feature_selection import SequentialFeatureSelector

# Load and preview datasets

In [14]:
#load required datasets
df = pd.read_csv('https://raw.githubusercontent.com/risenW/Practical_feature_engineering_guide/master/sendy_logistics.csv')
df.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,5,10:04:47 AM,9,5,10:27:30 AM,9,5,10:39:55 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16 AM,12,5,11:23:21 AM,12,5,11:40:22 AM,12,5,11:44:09 AM,12,5,12:17:22 PM,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,12:39:25 PM,30,2,12:42:44 PM,30,2,12:49:34 PM,30,2,12:53:03 PM,30,2,1:00:38 PM,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,9:25:34 AM,15,5,9:26:05 AM,15,5,9:37:56 AM,15,5,9:43:06 AM,15,5,10:05:27 AM,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,9:55:18 AM,13,1,9:56:18 AM,13,1,10:03:53 AM,13,1,10:05:23 AM,13,1,10:25:37 AM,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214


In [7]:
data_desc = pd.read_csv('https://raw.githubusercontent.com/WKhisa/Feature-Engineering-with-Python/main/VariableDefinitions.csv')
data_desc.head()

,Order No,Unique number identifying the order
0,User Id,Unique number identifying the customer on a pl...
1,Vehicle Type,"For this competition limited to bikes, however..."
2,Platform Type,"Platform used to place the order, there are 4 ..."
3,Personal or Business,Customer type
4,Placement - Day of Month,Placement - Day of Month i.e 1-31


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21201 entries, 0 to 21200
Data columns (total 29 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Order No                                   21201 non-null  object 
 1   User Id                                    21201 non-null  object 
 2   Vehicle Type                               21201 non-null  object 
 3   Platform Type                              21201 non-null  int64  
 4   Personal or Business                       21201 non-null  object 
 5   Placement - Day of Month                   21201 non-null  int64  
 6   Placement - Weekday (Mo = 1)               21201 non-null  int64  
 7   Placement - Time                           21201 non-null  object 
 8   Confirmation - Day of Month                21201 non-null  int64  
 9   Confirmation - Weekday (Mo = 1)            21201 non-null  int64  
 10  Confirmation - Time   

# Clean and standardize data

In [9]:
#Inspect Columns
df.columns

Index(['Order No', 'User Id', 'Vehicle Type', 'Platform Type',
       'Personal or Business', 'Placement - Day of Month',
       'Placement - Weekday (Mo = 1)', 'Placement - Time',
       'Confirmation - Day of Month', 'Confirmation - Weekday (Mo = 1)',
       'Confirmation - Time', 'Arrival at Pickup - Day of Month',
       'Arrival at Pickup - Weekday (Mo = 1)', 'Arrival at Pickup - Time',
       'Pickup - Day of Month', 'Pickup - Weekday (Mo = 1)', 'Pickup - Time',
       'Arrival at Destination - Day of Month',
       'Arrival at Destination - Weekday (Mo = 1)',
       'Arrival at Destination - Time', 'Distance (KM)', 'Temperature',
       'Precipitation in millimeters', 'Pickup Lat', 'Pickup Long',
       'Destination Lat', 'Destination Long', 'Rider Id',
       'Time from Pickup to Arrival'],
      dtype='object')

In [15]:
#drop unnecessary columns
drop_columns = ['Order No', 'User Id', 'Rider Id', 'Vehicle Type','Confirmation - Day of Month', 'Confirmation - Weekday (Mo = 1)','Confirmation - Time', 'Arrival at Pickup - Day of Month','Arrival at Pickup - Weekday (Mo = 1)', 'Arrival at Pickup - Time',
       'Pickup - Day of Month', 'Pickup - Weekday (Mo = 1)', 'Pickup - Time','Arrival at Destination - Day of Month','Arrival at Destination - Weekday (Mo = 1)']
df.drop(columns= drop_columns, axis = 1, inplace= True )
df.columns

Index(['Platform Type', 'Personal or Business', 'Placement - Day of Month',
       'Placement - Weekday (Mo = 1)', 'Placement - Time',
       'Arrival at Destination - Time', 'Distance (KM)', 'Temperature',
       'Precipitation in millimeters', 'Pickup Lat', 'Pickup Long',
       'Destination Lat', 'Destination Long', 'Time from Pickup to Arrival'],
      dtype='object')

In [16]:
#Standardize Column names
df.columns = df.columns.str.strip().str.lower().str.replace(" ","_")
df.columns

Index(['platform_type', 'personal_or_business', 'placement_-_day_of_month',
       'placement_-_weekday_(mo_=_1)', 'placement_-_time',
       'arrival_at_destination_-_time', 'distance_(km)', 'temperature',
       'precipitation_in_millimeters', 'pickup_lat', 'pickup_long',
       'destination_lat', 'destination_long', 'time_from_pickup_to_arrival'],
      dtype='object')

In [18]:
#chekc duplicates
df.duplicated().sum()

0

In [19]:
#check missing
df.isna().sum()

platform_type                        0
personal_or_business                 0
placement_-_day_of_month             0
placement_-_weekday_(mo_=_1)         0
placement_-_time                     0
arrival_at_destination_-_time        0
distance_(km)                        0
temperature                       4366
precipitation_in_millimeters     20649
pickup_lat                           0
pickup_long                          0
destination_lat                      0
destination_long                     0
time_from_pickup_to_arrival          0
dtype: int64

Kenya, being tropical doesnt have much variation in temperature, however, values without rainfall could infer when it did not rain. Thus replace missing precipitation columns with '0', while dropping temperature column

In [20]:
df['precipitation_in_millimeters'].fillna(0,inplace=True)

In [21]:
df.drop(columns = ['temperature'], inplace = True)
df.head()

,platform_type,personal_or_business,placement_-_day_of_month,placement_-_weekday_(mo_=_1),placement_-_time,arrival_at_destination_-_time,distance_(km),precipitation_in_millimeters,pickup_lat,pickup_long,destination_lat,destination_long,time_from_pickup_to_arrival
0,3,Business,9,5,9:35:46 AM,10:39:55 AM,4,0.0,-1.317755,36.830370,-1.300406,36.829741,745
1,3,Personal,12,5,11:16:16 AM,12:17:22 PM,16,0.0,-1.351453,36.899315,-1.295004,36.814358,1993
2,3,Business,30,2,12:39:25 PM,1:00:38 PM,3,0.0,-1.308284,36.843419,-1.300921,36.828195,455
3,3,Business,15,5,9:25:34 AM,10:05:27 AM,9,0.0,-1.281301,36.832396,-1.257147,36.795063,1341
4,1,Personal,13,1,9:55:18 AM,10:25:37 AM,9,0.0,-1.266597,36.792118,-1.295041,36.809817,1214


In [24]:
#categoical values
df['personal_or_business'] = df['personal_or_business'].astype('category').cat.codes


In [27]:
df['personal_or_business'].nunique()

2